<a href="https://colab.research.google.com/github/glitchheadgit/smilesChessFraud/blob/main/chessFraudTusyan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
2 + 2

4